In [1]:
!pip install openai==0.28.1
!pip install fpdf==1.7.2

  Using cached fpdf-1.7.2.tar.gz (39 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=78eefbbf3ad21b5a7d3cec6cd58d9cca4373e02e0e7f530be2ed551823d12218
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [ ]:
import pandas as pd
import openai
import time
from fpdf import FPDF
import os

openai.api_key = ' '
os.environ['OPENAI_API_KEY'] = ' '

## Load the dataset

In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/vaniebermudez/ai_first_bootcamp/refs/heads/main/Day4/inventory_products_dataset.csv')
df

,Product ID,Product Name,Category,SKU,Description,Supplier Name,Quantity in Stock,Reorder Level,Reorder Quantity,Unit Price,Total Inventory Value,Location,Date of Last Restock,Lead Time,Expiration Date,Batch Number,Condition,Barcode
0,1,Widget A,Hardware,WID123,High-quality metal widget,ABC Supplies,500,100,200,$5,$2500,Warehouse 1,11/01/2024,7 days,NaN,BATCH001,New,QR001
1,2,Bolt Pack,Fasteners,BOLT456,Set of 50 steel bolts,XYZ Fasteners,250,50,100,$2,$500,Warehouse 2,10/30/2024,5 days,NaN,BATCH002,New,QR002
2,3,Adhesive Tape,Supplies,TAP789,Industrial adhesive tape,TapeTech Inc.,150,30,50,$3,$450,Warehouse 3,11/05/2024,3 days,NaN,BATCH003,New,QR003
3,4,Safety Goggles,Safety Gear,SAF001,Protective eyewear,SafeWare Ltd.,75,20,50,$8,$600,Warehouse 1,11/02/2024,10 days,NaN,BATCH004,New,QR004
4,5,Welding Rods,Welding,WEL123,High-strength welding rods,WeldCo Supplies,120,40,80,$10,$1200,Warehouse 2,11/01/2024,15 days,NaN,BATCH005,New,QR005
5,6,Drill Bit Set,Tools,DRI456,10-piece drill bit set,ToolMasters Inc.,200,50,100,$15,$3000,Warehouse 3,10/29/2024,8 days,NaN,BATCH006,New,QR006
6,7,Electrical Wire,Electrical,ELE789,100m spool of copper wire,Electra Corp.,400,100,200,$20,$8000,Warehouse 1,11/04/2024,12 days,NaN,BATCH007,New,QR007
7,8,Hammer,Tools,HAM001,Heavy-duty hammer,HammerWorks Co.,90,20,50,$7,$630,Warehouse 3,11/03/2024,5 days,NaN,BATCH008,New,QR008
8,9,Paint Can (1L),Paints,PAI123,1-liter can of paint,ColorWorld,300,75,150,$12,$3600,Warehouse 2,11/01/2024,10 days,04/01/2025,BATCH009,New,QR009
9,10,Sandpaper Pack,Supplies,SAN456,Assorted sandpaper sheets,SmoothTech,500,100,250,$1,$500,Warehouse 1,11/06/2024,4 days,NaN,BATCH010,New,QR010


In [7]:
df.sample(5)

,Product ID,Product Name,Category,SKU,Description,Supplier Name,Quantity in Stock,Reorder Level,Reorder Quantity,Unit Price,Total Inventory Value,Location,Date of Last Restock,Lead Time,Expiration Date,Batch Number,Condition,Barcode
19,20,Cable Ties Pack,Electrical,CAB001,Pack of 100 cable ties,Electra Corp.,600,150,300,$1,$600,Warehouse 2,11/02/2024,5 days,NaN,BATCH020,New,QR020
18,19,Insulation Foam,Supplies,FOA789,Thermal insulation foam,FoamGuard Ltd.,400,100,200,$10,$4000,Warehouse 1,11/04/2024,7 days,01/15/2025,BATCH019,New,QR019
14,15,Screwdriver Set,Tools,SCR001,Precision screwdriver set,ToolHub Inc.,150,50,75,$9,$1350,Warehouse 3,11/03/2024,5 days,NaN,BATCH015,New,QR015
17,18,PVC Pipes (2m),Hardware,PVC456,2-meter PVC pipe,PipeFlow Ltd.,80,20,40,$15,$1200,Warehouse 3,10/28/2024,8 days,NaN,BATCH018,New,QR018
4,5,Welding Rods,Welding,WEL123,High-strength welding rods,WeldCo Supplies,120,40,80,$10,$1200,Warehouse 2,11/01/2024,15 days,NaN,BATCH005,New,QR005


## Understanding the Dataset

In [9]:
# Get Column Names and Data Types
column_info = df.dtypes.reset_index()
print(column_info, "\n")


                    index       0
0              Product ID   int64
1            Product Name  object
2                Category  object
3                     SKU  object
4             Description  object
5           Supplier Name  object
6       Quantity in Stock   int64
7           Reorder Level   int64
8        Reorder Quantity   int64
9              Unit Price  object
10  Total Inventory Value  object
11               Location  object
12   Date of Last Restock  object
13              Lead Time  object
14        Expiration Date  object
15           Batch Number  object
16              Condition  object
17                Barcode  object 



In [10]:
# Identify Categorical and Numerical Features
categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns

print("Categorical Features:", categorical_features)
print("Numerical Features:", numerical_features)

Categorical Features: Index(['Product Name', 'Category', 'SKU', 'Description', 'Supplier Name',
       'Unit Price', 'Total Inventory Value', 'Location',
       'Date of Last Restock', 'Lead Time', 'Expiration Date', 'Batch Number',
       'Condition', 'Barcode'],
      dtype='object')
Numerical Features: Index(['Product ID', 'Quantity in Stock', 'Reorder Level', 'Reorder Quantity'], dtype='object')


In [20]:
# Get Unique Values for Categorical Features with Limitation
def get_unique_values(col, max_display=10):
    uniques = df[col].unique().tolist()
    if len(uniques) > max_display:
        return uniques[:max_display] + ["..."]
    return uniques

unique_values = {col: get_unique_values(col) for col in categorical_features}

print("Unique Values for Categorical Features:")
for col, uniques in unique_values.items():
    print(f" - {col}: {uniques}")
print()

Unique Values for Categorical Features:
 - Product Name: ['Widget A', 'Bolt Pack', 'Adhesive Tape', 'Safety Goggles', 'Welding Rods', 'Drill Bit Set', 'Electrical Wire', 'Hammer', 'Paint Can (1L)', 'Sandpaper Pack', '...']
 - Category: ['Hardware', 'Fasteners', 'Supplies', 'Safety Gear', 'Welding', 'Tools', 'Electrical', 'Paints', 'Raw Material']
 - SKU: ['WID123', 'BOLT456', 'TAP789', 'SAF001', 'WEL123', 'DRI456', 'ELE789', 'HAM001', 'PAI123', 'SAN456', '...']
 - Description: ['High-quality metal widget', 'Set of 50 steel bolts', 'Industrial adhesive tape', 'Protective eyewear', 'High-strength welding rods', '10-piece drill bit set', '100m spool of copper wire', 'Heavy-duty hammer', '1-liter can of paint', 'Assorted sandpaper sheets', '...']
 - Supplier Name: ['ABC Supplies', 'XYZ Fasteners', 'TapeTech Inc.', 'SafeWare Ltd.', 'WeldCo Supplies', 'ToolMasters Inc.', 'Electra Corp.', 'HammerWorks Co.', 'ColorWorld', 'SmoothTech', '...']
 - Unit Price: ['$5', '$2', '$3', '$8', '$10', '$15

In [16]:
# Get Descriptive Statistics for Numerical Features

numerical_stats = df[numerical_features].describe()
print(numerical_stats)

       Product ID  Quantity in Stock  Reorder Level  Reorder Quantity
count    20.00000          20.000000      20.000000         20.000000
mean     10.50000         238.750000      57.750000        115.250000
std       5.91608         164.699687      36.937892         77.891203
min       1.00000          60.000000      20.000000         30.000000
25%       5.75000          97.500000      27.500000         50.000000
50%      10.50000         190.000000      50.000000         90.000000
75%      15.25000         325.000000      85.000000        162.500000
max      20.00000         600.000000     150.000000        300.000000


## Generating the Template

In [24]:
def generate_template(df,
                             column_info,
                             categorical_features,
                             numerical_features,
                             unique_values,
                             numerical_stats):

    # Construct a summary of the dataframe's structure
    column_summary = "Column Names and Data Types: \n"
    for col, dtype in zip(df.columns, df.dtypes):
        column_summary += f"{col}: {dtype}\n"

    # Unique values for categorical features
    unique_values_str = "Unique Values for Categorical Features: \n"
    for col, uniques in unique_values.items():
        unique_values_str += f" - {col}: {uniques}\n"


    # Descriptive statistics for numerical features
    numerical_stats_str = "Descriptive Statistics for Numerical Features: \n"
    for col in numerical_features:
        numerical_stats_str += f" - {col}:\n"
        for stat_name, value in numerical_stats[col].items():
            numerical_stats_str += f"   {stat_name}: {value}\n"

    # Define the system prompt
    system_prompt = """
    You are an intelligent assistant that creates descriptive templates for transforming dataframe rows into coherent paragraphs.
    Analyze the provided dataframe structure and generate a template sentence that includes placeholders for each column.
    Ensure the template is contextually relevant and maintains grammatical correctness
    """

    # Define the user prompt
    user_prompt = f"""
    Analyze the following dataframe structure and create a descriptive template with placeholders for each column.

    <column_summary>
    {column_summary}
    </column_summary>

    <unique_values>
    {unique_values_str}
    </unique_values>

    <numerical_stats>
    {numerical_stats_str}
    </numerical_stats>

    Use the exact column names from the column_summary in generating the variable names in the template,
    as they will be populated with the actual values in the dataset.

    Example Template about a Spotify dataset:
    "{{artist}} gained {{streams}} streams in the song '{{song}}' that was a hit in {{date}}."


Output only the template without any explanation or introduction.
The template's variables will be dynamically replaced so make sure they're formatted properly."""


    # Generate the template (with retries)
    system_prompt = """You are an intelligent assistant that creates descriptive templates for transforming dataframe rows into coherent paragraphs.
    Analyze the provided dataframe structure and generate a template sentence that includes placeholders for each column.
    Ensure the template is contextually relevant and maintains grammatical correctness."""

    retries = 3
    for attempt in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                temperature=0.3,
                max_tokens=1024,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            )
            template = response['choices'][0]['message']['content'].strip()
            return template
        except Exception as e:
            print(f"Error generating template (Attempt {attempt + 1}/{retries}): {e}")
            time.sleep(2)  # Wait before retrying

    return None



In [25]:
# Generate the template
template = generate_template(df,
                             column_info,
                             categorical_features,
                             numerical_features,
                             unique_values,
                             numerical_stats)
print(template)


"The product '{Product Name}' (SKU: {SKU}), categorized under '{Category}', is supplied by '{Supplier Name}'. It has a total quantity of {Quantity in Stock} in stock, with a reorder level set at {Reorder Level} and a reorder quantity of {Reorder Quantity}. Priced at {Unit Price}, the total inventory value amounts to {Total Inventory Value}. This product is located in {Location} and was last restocked on {Date of Last Restock}. The lead time for this product is {Lead Time}, and it is in '{Condition}' condition. The expiration date is {Expiration Date}, and it belongs to batch number {Batch Number} with barcode {Barcode}."


## Populating the Template with Actual Values

In [27]:
def populate_template(template, row):

    # Convert row to dictionary and replace NaN with 'N/A'
    row_dict = row.to_dict()
    for key, value in row_dict.items():
      if pd.isna(value):
        row_dict[key] = 'N/A'
    # Generate the populated template per row

    paragraph = template.format(**row_dict)

    return paragraph



In [28]:
# Apply the populate_template function to each row in the dataframe
df['paragraph'] = df.apply(lambda row: populate_template(template, row), axis=1)


In [31]:
df['paragraph'][0]

'"The product \'Widget A\' (SKU: WID123), categorized under \'Hardware\', is supplied by \'ABC Supplies\'. It has a total quantity of 500 in stock, with a reorder level set at 100 and a reorder quantity of 200. Priced at $5, the total inventory value amounts to $2500. This product is located in Warehouse 1 and was last restocked on 11/01/2024. The lead time for this product is 7 days, and it is in \'New\' condition. The expiration date is N/A, and it belongs to batch number BATCH001 with barcode QR001."'

In [32]:
df.sample(5)

,Product ID,Product Name,Category,SKU,Description,Supplier Name,Quantity in Stock,Reorder Level,Reorder Quantity,Unit Price,Total Inventory Value,Location,Date of Last Restock,Lead Time,Expiration Date,Batch Number,Condition,Barcode,paragraph
4,5,Welding Rods,Welding,WEL123,High-strength welding rods,WeldCo Supplies,120,40,80,$10,$1200,Warehouse 2,11/01/2024,15 days,NaN,BATCH005,New,QR005,"""The product 'Welding Rods' (SKU: WEL123), cat..."
8,9,Paint Can (1L),Paints,PAI123,1-liter can of paint,ColorWorld,300,75,150,$12,$3600,Warehouse 2,11/01/2024,10 days,04/01/2025,BATCH009,New,QR009,"""The product 'Paint Can (1L)' (SKU: PAI123), c..."
7,8,Hammer,Tools,HAM001,Heavy-duty hammer,HammerWorks Co.,90,20,50,$7,$630,Warehouse 3,11/03/2024,5 days,NaN,BATCH008,New,QR008,"""The product 'Hammer' (SKU: HAM001), categoriz..."
12,13,Wrench Set,Tools,WRE789,8-piece wrench set,FixIt Tools,70,20,50,$18,$1260,Warehouse 1,11/04/2024,14 days,NaN,BATCH013,New,QR013,"""The product 'Wrench Set' (SKU: WRE789), categ..."
1,2,Bolt Pack,Fasteners,BOLT456,Set of 50 steel bolts,XYZ Fasteners,250,50,100,$2,$500,Warehouse 2,10/30/2024,5 days,NaN,BATCH002,New,QR002,"""The product 'Bolt Pack' (SKU: BOLT456), categ..."


## Saving the Populated Templates (Natural Language Texts)

In [36]:
def save_content_to_txt(paragraphs, txt_filename):
    try:
        with open(txt_filename, 'w', encoding='utf-8') as file:

            for para in paragraphs:
                file.write(para + '\n\n')  # Double newline for separation

        print(f"All content has been saved as '{txt_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to text file: {e}")


In [35]:
def save_content_to_pdf(paragraphs, pdf_filename):
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.set_font("Arial", size=12)

        # Add each paragraph
        for para in paragraphs:
            pdf.multi_cell(0, 10, para)
            pdf.ln()  # Add a line break between paragraphs

        pdf.output(pdf_filename)
        print(f"All content has been saved as '{pdf_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to PDF: {e}")

In [37]:
# Save the paragraphs to both text and PDF files
save_content_to_txt(df['paragraph'].to_list(), 'invy_nlg.txt')
save_content_to_pdf(df['paragraph'].to_list(), 'invy_nlg.pdf')

All content has been saved as 'invy_nlg.txt'.

All content has been saved as 'invy_nlg.pdf'.

